In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import fetch

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in sorted(filenames):
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
worlddata = pd.read_csv('/kaggle/input/john-csse-clone/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',
           index_col='Country/Region')
worlddata.head(15)

In [ ]:
worlddata = worlddata.groupby('Country/Region').sum()
worlddata.drop(['Lat','Long'], axis=1, inplace=True)
worlddata.head(10)

In [ ]:
isoformat = pd.read_csv('/kaggle/input/countries-data/countries.csv', usecols=['iso_alpha3','ccse_name'])
isoformat.head()

In [ ]:
worlddata = isoformat.set_index('ccse_name').join(worlddata, how='inner')
worlddata

In [ ]:
last_day = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')
last_day = last_day[['Last_Update','Confirmed','Deaths','ISO3']]
last_day.head()

In [ ]:
worlddata_full = worlddata.set_index('iso_alpha3').join(last_day.set_index('ISO3'))
worlddata_full.head()

In [ ]:
from datetime import datetime
today = datetime.today().strftime('%-m/%-d/%y')
worlddata_full[today] = worlddata_full['Confirmed']
worlddata_full.drop(['Last_Update','Deaths','Confirmed'], axis=1, inplace=True)
worlddata_full.head()

In [ ]:
#Получили итоговые данные по странам на текущую дату
countries = worlddata_full.T
countries.tail()

In [ ]:
data = fetch.fetch_yandex(dump_folder='')

In [ ]:
data, filepath = fetch.format_csse2(data, dump_folder='')

In [ ]:
#Данные для российских регионов перестали обновляться с 31.05.2020, нужно будет вбивать вручную
russiadata = pd.read_csv('https://raw.githubusercontent.com/grwlf/COVID-19_plus_Russia/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_RU.csv')
russiadata

russia_latest = pd.read_csv(filepath)
russia_latest.head()
russia = russiadata.set_index('Province_State').join(russia_latest.set_index('Province_State')['Confirmed'])
russia.head()

russia.drop(['UID','iso2','iso3','FIPS','Admin2','Country_Region','Lat','Long_','Combined_Key','code3'], axis=1, inplace=True)
russia[today] = russia['Confirmed']
del russia['Confirmed']
russia.head()

russia = russia.T
russia.tail()

In [ ]:
#Посмотрим какие регионы похожи на Новосибирскую область в терминах городской популяции, дополним еще схожими по плотности населения
russia_regions = pd.read_csv('/kaggle/input/russia-regions-in-sber-covid-competition/russia_regions.csv')
russia_regions.head()

In [ ]:
#Отсортируем по популяции
russia_regions_sorted = russia_regions.sort_values(by='population_urban',ascending=False)
russia_regions_sorted['index'] = np.arange(len(russia_regions))
russia_regions_sorted = russia_regions_sorted.set_index('index')
russia_regions_sorted

In [ ]:
nsk = russia_regions_sorted.loc[russia_regions_sorted['csse_province_state'] == 'Novosibirsk oblast']
nsk

In [ ]:
#Выберем по два региона в обе стороны
regionsalike = russia_regions_sorted.iloc[[12,13,14,15,16],[4]]
regionsalike

In [ ]:
#Добавим Москву, Санкт Петербург (как эталонные регионы) и Алтайский край (с похожей плотностью)
russia_needed = russia[['Moscow','Kemerovo oblast','Krasnoyarskiy kray','Novosibirsk oblast','Perm oblast','Volgograd oblast','Saint Petersburg','Altayskiy kray']]

In [ ]:
#Сохраним данные, и добавим недостающие даты для нужных нам регионов
russia_needed.to_excel('russia_needed.xlsx')

In [ ]:
#Добавили недостающие данные, загрузили
russia_similar = pd.read_excel('/kaggle/input/similar-regions/russia_needed.xlsx')
russia_similar.tail()

In [ ]:
indexcol = russia_similar.columns.values
indexcol[0] = 'Date'
russia_similar.columns = indexcol
russia_similar = russia_similar.set_index('Date')
russia_similar

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(15,10))

In [ ]:
#Построим график для ТОП-10, чтобы не перегружать, общий тренд и разницы будут видны
TOP_N = 10

top_countries = list(countries.loc[today].sort_values()[-TOP_N:].index)
top_countries

In [ ]:
top_countries = countries[['DEU', 'FRA', 'PER', 'ITA', 'ESP', 'IND', 'GBR', 'RUS', 'BRA', 'USA']]
top_countries.tail()

In [ ]:
#График для абсолютных показателей
plt.figure(figsize=(15,10))
plt.plot(top_countries)
plt.legend(['DEU',"FRA","PER","ITA","ESP","IND","GBR","RUS","BRA","USA"])

plt.show()

In [ ]:
#График для логарифмов - в среднем ~ на 7 точка перегиба, потом рост начинает затухать
plt.figure(figsize=(15,10))
plt.plot(np.log(top_countries+0.5))
plt.legend(['DEU',"FRA","PER","ITA","ESP","IND","GBR","RUS","BRA","USA"])
plt.show()

In [ ]:
#Посмотрим, что с нашими регионами
russia_similar.columns = ['Moscow', 'Kemerovo','Krasnoyarsk','Novosibirsk','Perm','Volgograd','SaintP','Altay Kray']
plt.figure(figsize=(15,10))
plt.plot(russia_similar)
plt.legend(['Moscow', 'Kemerovo','Krasnoyarsk','Novosibirsk','Perm','Volgograd','SaintP','Altay Kray'])
plt.show()

In [ ]:
#Преобразуем в логарифмы
plt.figure(figsize=(15,10))
plt.plot(np.log(russia_similar+0.5))
plt.legend(['Moscow', 'Kemerovo','Krasnoyarsk','Novosibirsk','Perm','Volgograd','SaintP','Altay Kray'])
plt.show()

In [ ]:
#Только Новосибирская область
plt.figure(figsize=(15,10))
plt.plot(russia_similar['Novosibirsk']+0.5)
plt.show()

In [ ]:
#Только Новосибирская область в логарифмах
plt.figure(figsize=(15,10))
plt.plot(np.log(russia_similar['Novosibirsk']+0.5))
plt.show()

In [ ]:
#Возьмем первые разности
russia_similar_df1 = russia_similar.diff()
russia_similar_df1

In [ ]:
#построим первые разности (без Москвы)
plt.figure(figsize=(15,10))
regionsnomoscow = russia_similar_df1.drop(['Moscow'],axis =1)
plt.plot(regionsnomoscow)
plt.legend(['Kemerovo','Krasnoyarsk','Novosibirsk','Perm','Volgograd','SaintP','Altay Kray'])
plt.show()

In [ ]:
nsk = russia_similar.drop(['Moscow', 'Kemerovo','Krasnoyarsk','Perm','Volgograd','SaintP','Altay Kray'],axis =1)
nsk

In [ ]:
nsk0 = nsk.loc[(nsk!=0).any(axis=1)]
nsk0

In [ ]:
nsk0_df1 = nsk0.diff()

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(nsk0, 'g-')
ax2.plot(nsk0_df1, 'b-')
plt.plot()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit
from scipy.optimize import fsolve

In [ ]:
nsk1 = nsk0.reset_index()
nsk2 = nsk1.reset_index()
nsk2
nskdata = nsk2.drop(['Date'],axis=1)
nskdata

In [ ]:
def logistic_model(x,a,b,c):
    return c/(1+np.exp(-(x-b)/a))

In [ ]:
x = list(nskdata.iloc[:,0])
y = list(nskdata.iloc[:,1])
fit = curve_fit(logistic_model,x,y)

In [ ]:
fit

In [ ]:
errors = [np.sqrt(fit[1][i][i]) for i in [0,1,2]]

In [ ]:
errors

In [ ]:
a = fit[0][0]
b = fit[0][1]
c = fit[0][2]

In [ ]:
sol = int(fsolve(lambda x : logistic_model(x,a,b,c) - int(c),b))

In [ ]:
sol

In [ ]:
pred_x = list(range(max(x),sol))
pred_x

In [ ]:
plt.figure(figsize=(15,10))
plt.scatter(x,y,label="Real data",color="red")
plt.plot(x+pred_x, [logistic_model(i,fit[0][0],fit[0][1],fit[0][2]) for i in x+pred_x], label="Logistic model",color='green' )
plt.legend()
plt.show()

In [ ]:
nsk1_df1 = nsk0_df1.reset_index()
nsk2_df1 = nsk1_df1.reset_index()
nsk2_df1
nskdata_df1 = nsk2_df1.drop(['Date'],axis=1)
nskdata_df1
nskdata_df1 

In [ ]:
def gaussian_model(x, a, b, c):
    y = a * np.exp(-0.5 * ((x-b)/c)**2)
    return y

In [ ]:
nskdata_df1.drop([0], inplace=True)

In [ ]:
nskdata_df1

In [ ]:
ax = nskdata_df1.plot.bar(x='index', y='Novosibirsk', rot=0)

In [ ]:
x = list(nskdata_df1.iloc[:,0])
y = list(nskdata_df1.iloc[:,1])
fit = curve_fit(gaussian_model,x,y)

In [ ]:
fit

In [ ]:
pred_x = list(range(max(x),150))
pred_x

In [ ]:
plt.figure(figsize=(15,10))
plt.bar(x,y,label="Real data",color="red")
plt.plot(x+pred_x, [gaussian_model(i,fit[0][0],fit[0][1],fit[0][2]) for i in x+pred_x], label="Gaussian model",color='green' )
plt.legend()
plt.show()